In [1]:
import pandas as pd
from fastpandas import FastPandas

In [2]:
df = pd.DataFrame({"a": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], "b": [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]})

In [3]:
# lazy eval
out = FastPandas(df)["a"].avg()
print(out)

# nothing has been computed yet!
# now when you access the item, it will be computed
print(out.item())

avg("a")
5.5


In [4]:
import random
# you can choose from the aggregate or numeric functions found on DuckDB
large_df = pd.DataFrame({"a": random.choices(range(35_000_000), k=100_000_000)})

In [5]:
unique_elements = FastPandas(large_df)["a"].approx_count_distinct().item()
print(unique_elements) # uses hyperloglog under the hooda

32981280


In [6]:
# that was on a 100 million row dataframe in much less than a second!!!
print(f"{len(large_df):,}")

100,000,000


In [7]:
# correlation between two columns for 100 million rows???
large_df["b"] = FastPandas(large_df)["a"].abs().add(1).ln().df() # some random other column

# a is y, b is x
print(FastPandas(large_df)["a"].corr(FastPandas(large_df)["b"]).item())

0.8660407741494259


In [8]:
# filtering too!
output = FastPandas(large_df)["a"].filter(FastPandas(large_df)["a"].gt(5)).avg()
print(output.item())

17498532.055985566


In [9]:
FastPandas(large_df)["a"].filter(FastPandas(large_df)["a"].gt(0)._and(FastPandas(large_df)["b"].lt(10))).avg().item()

10965.912107082819

In [10]:
# count distinct between 0 and 10
between_0_and_10 = FastPandas(large_df)["a"].gte(0)._and(FastPandas(large_df)["b"].lte(10))
FastPandas(large_df)["a"].filter(between_0_and_10).approx_count_distinct().item()

21050

In [12]:
# type casting
_factorial = FastPandas(df)["a"].factorial()
_factorial.df()

BinderException: Binder Error: No function matches the given name and argument types 'factorial(BIGINT)'. You might need to add explicit type casts.
	Candidate functions:
	factorial(INTEGER) -> HUGEINT


In [16]:
_factorial = FastPandas(df)["a"].int().factorial()
_factorial.df()

,factorial(CAST(a AS INTEGER))
0,1.0
1,2.0
2,6.0
3,24.0
4,120.0
5,720.0
6,5040.0
7,40320.0
8,362880.0
9,3628800.0
